In [ ]:
## how might one format a columnar table?
##  - this notebook is only meant to explore one possible, simple method and how an end to end solution may look.

In [2]:
import numpy as np

In [3]:
input_text = """

  right       10
  forward     m/s
  velocity
  left        50
  forward     m/s
  velocity
  speed       .5
  ratio

"""

output_text = """

right forward velocity 10 m/s
left forward velocity 50 m/s
speed ratio .5

"""

In [4]:
from nltk.tokenize import word_tokenize

In [5]:
from collections import defaultdict

In [6]:
freqs = defaultdict(int)

freqs['<>'] = 0

for c in input_text:
    freqs[c] += 1

In [7]:
vocab = list(freqs.keys())

n_vocab = len(vocab)

letter_to_index = dict([ (w, i) for i, w in enumerate(vocab) ])
index_to_letter = dict([ (i, w) for i, w in enumerate(vocab) ])

print(vocab)

['<>', '\n', ' ', 'r', 'i', 'g', 'h', 't', '1', '0', 'f', 'o', 'w', 'a', 'd', 'm', '/', 's', 'v', 'e', 'l', 'c', 'y', '5', 'p', '.']


In [8]:
import re

In [9]:
def translate(text: str, letter_to_index: dict):
    return [ letter_to_index[key] for key in text ]

def translate_output(text: str, letter_to_index: dict, size: int):
    a = translate(text, letter_to_index)
    
    ## pad,
    n = len(a)
    if n < size:
        p = size - n
        a = a + [ letter_to_index['<>'] for _ in range(p) ]
    
    return a

In [10]:
input_vector = np.array(translate(input_text, letter_to_index))
input_vector

array([ 1,  1,  2,  2,  3,  4,  5,  6,  7,  2,  2,  2,  2,  2,  2,  2,  8,
        9,  1,  2,  2, 10, 11,  3, 12, 13,  3, 14,  2,  2,  2,  2,  2, 15,
       16, 17,  1,  2,  2, 18, 19, 20, 11, 21,  4,  7, 22,  1,  2,  2, 20,
       19, 10,  7,  2,  2,  2,  2,  2,  2,  2,  2, 23,  9,  1,  2,  2, 10,
       11,  3, 12, 13,  3, 14,  2,  2,  2,  2,  2, 15, 16, 17,  1,  2,  2,
       18, 19, 20, 11, 21,  4,  7, 22,  1,  2,  2, 17, 24, 19, 19, 14,  2,
        2,  2,  2,  2,  2,  2, 25, 23,  1,  2,  2,  3, 13,  7,  4, 11,  1,
        1])

In [11]:
output_vector = np.array(translate_output(output_text, letter_to_index, len(input_vector)))
output_vector

array([ 1,  1,  3,  4,  5,  6,  7,  2, 10, 11,  3, 12, 13,  3, 14,  2, 18,
       19, 20, 11, 21,  4,  7, 22,  2,  8,  9,  2, 15, 16, 17,  1, 20, 19,
       10,  7,  2, 10, 11,  3, 12, 13,  3, 14,  2, 18, 19, 20, 11, 21,  4,
        7, 22,  2, 23,  9,  2, 15, 16, 17,  1, 17, 24, 19, 19, 14,  2,  3,
       13,  7,  4, 11,  2, 25, 23,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0])

In [12]:
assert len(input_vector) == len(output_vector)

print(len(input_vector))

120


In [13]:
num_encoder_tokens = len(input_vector)
num_decoder_tokens = len(output_vector)

In [14]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, LSTM, TimeDistributed, Dense, Bidirectional, Dropout

Using TensorFlow backend.


In [15]:
layers = [ 100, 100 ]

In [16]:
### https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py

model = Sequential()

model.add(Bidirectional(LSTM(100, input_shape=(len(input_vector), n_vocab), return_sequences=True)))
model.add(Dropout(0.2))

for layer in layers:
    model.add(LSTM(layer, return_sequences=True))
    model.add(Dropout(0.2))

## apply dense network to each t
model.add(TimeDistributed(Dense(n_vocab, activation='softmax')))

In [17]:
X = np.array([to_categorical(input_vector.tolist(), n_vocab)])
y = np.array([to_categorical(output_vector, n_vocab)])

## output should be probability distribution, (softmax, one-hot)
print(y[0][5])

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [18]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics = ['accuracy']
)

## fire away!!!!, over-train!!!
model.fit(X, y, epochs=500)

Epoch 1/500
1/1 [==============================] - 1s 1s/step - loss: 3.2544 - accuracy: 0.0833
Epoch 2/500
1/1 [==============================] - 0s 227ms/step - loss: 3.1984 - accuracy: 0.3583
Epoch 3/500
1/1 [==============================] - 0s 207ms/step - loss: 3.1276 - accuracy: 0.3583
Epoch 4/500
1/1 [==============================] - 0s 251ms/step - loss: 3.0019 - accuracy: 0.3583
Epoch 5/500
1/1 [==============================] - 0s 250ms/step - loss: 2.8701 - accuracy: 0.3583
Epoch 6/500
1/1 [==============================] - 0s 198ms/step - loss: 2.6700 - accuracy: 0.3583
Epoch 7/500
1/1 [==============================] - 0s 191ms/step - loss: 2.7116 - accuracy: 0.3583
Epoch 8/500
1/1 [==============================] - 0s 225ms/step - loss: 2.7279 - accuracy: 0.3583
Epoch 9/500
1/1 [==============================] - 0s 204ms/step - loss: 2.7127 - accuracy: 0.3583
Epoch 10/500
1/1 [==============================] - 0s 217ms/step - loss: 2.6382 - accuracy: 0.3583
Epoch 11/500

In [19]:
prediction = [
    np.argmax(x)
    for x 
    in model.predict(X)[0]
]

In [20]:
print(input_text)



  right       10
  forward     m/s
  velocity
  left        50
  forward     m/s
  velocity
  speed       .5
  ratio




In [21]:
predicted_output_text = re.sub(r'<>', '', ''.join([ index_to_letter[i] for i in prediction ]))
predicted_output_text

'\n\nright forward velocity 10 m/s\nleft forward velocity 50 m/s\nspeed ratio .5\n\n'

In [22]:
## flattened columns,
print(predicted_output_text)



right forward velocity 10 m/s
left forward velocity 50 m/s
speed ratio .5


